In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
# --- Installation Forcée des Bibliothèques ---
# Cette cellule garantit que nous utilisons les bonnes versions des bibliothèques.
print("Installation des versions spécifiques de transformers, datasets, accelerate et peft...")
os.system("pip install --quiet transformers==4.30.2 datasets==2.12.0 accelerate==0.20.3 peft==0.4.0")
print("Installation terminée.")

Installation des versions spécifiques de transformers, datasets, accelerate et peft...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.17.6 requires transformers>=4.33.1, but you have transformers 4.30.2 which is incompatible.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 which is incompatible.


Installation terminée.


In [3]:
pip install shutil

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import os
import shutil

2026-02-15 13:48:12.310293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771163292.714449      21 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771163292.835822      21 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# --- Configuration ---
# Définissez le nom du modèle que vous souhaitez utiliser depuis le Hub Hugging Face.
# DeBERTa-v3-base est un excellent point de départ.
MODEL_NAMES = [
    "microsoft/deberta-v3-base",
    "roberta-base" # Décommentez pour essayer un deuxième modèle
]

In [6]:
# Définissez les noms de vos fichiers. Assurez-vous que ces fichiers se trouvent dans le même répertoire.
TRAIN_CSV_PATH = "/kaggle/input/llm-classification-finetuning/train.csv"
TEST_CSV_PATH = "/kaggle/input/llm-classification-finetuning/test.csv"

# --- 1. Chargement des Données ---
print("Étape 1: Chargement des données...")
try:
    train_df = pd.read_csv(TRAIN_CSV_PATH)
    test_df = pd.read_csv(TEST_CSV_PATH)
    print(f"Données d'entraînement chargées: {train_df.shape[0]} lignes")
    print(f"Données de test chargées: {test_df.shape[0]} lignes")
except FileNotFoundError as e:
    print(f"Erreur: {e}. Assurez-vous que les fichiers CSV sont dans le bon répertoire.")
    exit()

Étape 1: Chargement des données...
Données d'entraînement chargées: 57477 lignes
Données de test chargées: 3 lignes


In [7]:
# --- 2. Prétraitement et Division des Données ---
print("\nÉtape 2: Prétraitement et division des données...")

def create_input_text(row):
    return f"PROMPT: {row['prompt']} [SEP] RESPONSE A: {row['response_a']} [SEP] RESPONSE B: {row['response_b']}"

train_df['text'] = train_df.apply(create_input_text, axis=1)
test_df['text'] = test_df.apply(create_input_text, axis=1)

target_cols = ['winner_model_a', 'winner_model_b', 'winner_tie']
train_df['label'] = np.argmax(train_df[target_cols].values, axis=1)

# Diviser les données d'entraînement en un ensemble d'entraînement et un ensemble de validation
train_subset_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

print(f"Taille du sous-ensemble d'entraînement: {len(train_subset_df)}")
print(f"Taille de l'ensemble de validation: {len(eval_df)}")

id2label = {0: "winner_model_a", 1: "winner_model_b", 2: "winner_tie"}
label2id = {"winner_model_a": 0, "winner_model_b": 1, "winner_tie": 2}


Étape 2: Prétraitement et division des données...
Taille du sous-ensemble d'entraînement: 45981
Taille de l'ensemble de validation: 11496


In [8]:
# --- Boucle d'entraînement pour plusieurs modèles ---
best_model_path = None
best_logloss = float('inf')
results = {}

for model_name in MODEL_NAMES:
    print(f"\n--- Début du processus pour le modèle: {model_name} ---")

    # --- 3. Préparation pour le Modèle ---
    print(f"\nÉtape 3: Initialisation du Tokenizer pour '{model_name}'...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_dataset = Dataset.from_pandas(train_subset_df[['text', 'label']])
    eval_dataset = Dataset.from_pandas(eval_df[['text', 'label']])
    test_dataset = Dataset.from_pandas(test_df[['text']])

    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")

    print("Tokenisation des datasets...")
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # --- 4. Définition et Entraînement du Modèle ---
    print(f"\nÉtape 4: Chargement du modèle pré-entraîné '{model_name}'...")
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3,
        id2label=id2label,
        label2id=label2id
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        probs = torch.nn.functional.softmax(torch.from_numpy(predictions), dim=-1).numpy()
        return {"logloss": log_loss(labels, probs)}

    output_dir = f"./results_{model_name.replace('/', '_')}"
    print("Définition des arguments d'entraînement...")
    # CORRECTION: Utilisation des arguments modernes compatibles avec la version installée
    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",       # Évalue à la fin de chaque époque
        save_strategy="epoch",             # Sauvegarde à la fin de chaque époque
        load_best_model_at_end=True,       # Charger le meilleur modèle à la fin
        metric_for_best_model="logloss",   # Métrique pour déterminer le meilleur modèle
        greater_is_better=False,           # Pour logloss, une valeur plus faible est meilleure
        push_to_hub=False,
        fp16=torch.cuda.is_available(),
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("\nDébut de l'entraînement...")
    trainer.train()
    print("Entraînement terminé !")

    # Évaluation et stockage des résultats
    eval_results = trainer.evaluate()
    current_logloss = eval_results['eval_logloss']
    results[model_name] = current_logloss
    print(f"LogLoss de validation pour {model_name}: {current_logloss}")

    if current_logloss < best_logloss:
        best_logloss = current_logloss
        best_model_path = trainer.state.best_model_checkpoint
        print(f"Nouveau meilleur modèle trouvé: {model_name} avec LogLoss: {best_logloss}")


--- Début du processus pour le modèle: microsoft/deberta-v3-base ---

Étape 3: Initialisation du Tokenizer pour 'microsoft/deberta-v3-base'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Logloss
1,1.082800,1.061497,1.061497


Entraînement terminé !


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


LogLoss de validation pour roberta-base: 1.0614971025344127
Nouveau meilleur modèle trouvé: roberta-base avec LogLoss: 1.0614971025344127


In [9]:
# --- 5. Prédiction avec le Meilleur Modèle ---
print("\n--- Processus d'entraînement terminé ---")
print("Résultats finaux:")
for model_name, score in results.items():
    print(f"- {model_name}: {score}")

if best_model_path is None:
    print("Aucun modèle n'a été entraîné avec succès. Sortie.")
    exit()

print(f"\nLe meilleur modèle est stocké dans: {best_model_path}")
print(f"Meilleur LogLoss de validation: {best_logloss}")


--- Processus d'entraînement terminé ---
Résultats finaux:
- microsoft/deberta-v3-base: 1.0775359059698186
- roberta-base: 1.0614971025344127

Le meilleur modèle est stocké dans: ./results_roberta-base/checkpoint-2874
Meilleur LogLoss de validation: 1.0614971025344127


In [10]:
print("\nÉtape 5: Prédiction sur l'ensemble de test avec le meilleur modèle...")
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
# Nous avons besoin d'un nouveau Trainer pour la prédiction
prediction_args = TrainingArguments(output_dir="./pred_results", per_device_eval_batch_size=16, report_to="none")
prediction_trainer = Trainer(model=best_model, args=prediction_args, data_collator=data_collator)

predictions = prediction_trainer.predict(tokenized_test_dataset)

logits = torch.from_numpy(predictions.predictions)
probabilities = torch.nn.functional.softmax(logits, dim=-1).numpy()

print("Formatage du fichier de soumission...")
submission_output = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': probabilities[:, 0],
    'winner_model_b': probabilities[:, 1],
    'winner_tie': probabilities[:, 2]
})

submission_output.to_csv("submission.csv", index=False)

# Nettoyage des dossiers de checkpoints pour économiser de l'espace
print("\nNettoyage des répertoires de travail...")
for model_name in MODEL_NAMES:
    dir_to_remove = f"./results_{model_name.replace('/', '_')}"
    if os.path.exists(dir_to_remove):
        shutil.rmtree(dir_to_remove)
if os.path.exists("./pred_results"):
    shutil.rmtree("./pred_results")

print("\nFichier 'submission.csv' créé avec succès !")
print("Aperçu de la soumission:")
print(submission_output.head())
print("\nLe script est terminé.")


Étape 5: Prédiction sur l'ensemble de test avec le meilleur modèle...


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:416: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Formatage du fichier de soumission...

Nettoyage des répertoires de travail...

Fichier 'submission.csv' créé avec succès !
Aperçu de la soumission:
        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.185441        0.232539    0.582020
1   211333        0.333516        0.395179    0.271304
2  1233961        0.388967        0.328058    0.282975

Le script est terminé.
